## This nb assisted with the files resulting from chunked detection (mostly for detectgpt)

### Note: this script should be moved to/run from the same dir as the `utils` subdir lives in to work properly

In [ ]:
import os
from argparse import Namespace
from tqdm import tqdm
from statistics import mean

import numpy as np
import pandas as pd
import torch

from datasets import Dataset

from utils.io import read_json, read_jsonlines, write_lst_json, write_jsonlines, write_json

from utils.notebooks import filter_text_col_length

In [5]:
# unclear whether the fulls can be eval'd on the end of the list successfully, may have to remove
# INPUT_DIR = OUTPUT_DIR = "/cmlscratch/jkirchen/watermarking-root/eval_groups/baselines/core/detectgpt/attacked/core_simple_1_50_200_no_wm_dipper_high"
# INPUT_DIR = OUTPUT_DIR = "/cmlscratch/jkirchen/watermarking-root/eval_groups/baselines/core/detectgpt/attacked/core_simple_1_50_200_no_wm_gpt_p4"
# INPUT_DIR = OUTPUT_DIR = "/cmlscratch/jkirchen/watermarking-root/eval_groups/baselines/core/detectgpt/attacked/core_simple_1_200_1000_no_wm_dipper_high"
# INPUT_DIR = OUTPUT_DIR = "/cmlscratch/jkirchen/watermarking-root/eval_groups/baselines/core/detectgpt/attacked/core_simple_1_200_1000_no_wm_gpt_p4"

# INPUT_DIR = OUTPUT_DIR = "/cmlscratch/jkirchen/watermarking-root/eval_groups/baselines/core/detectgpt/attacked/core_simple_1_200_1000_gen_cp_attack_1-25%"
INPUT_DIR = OUTPUT_DIR = "/cmlscratch/jkirchen/watermarking-root/eval_groups/baselines/core/detectgpt/attacked/core_simple_1_200_1000_gen_cp_attack_3-25%"

# INPUT_DIR = OUTPUT_DIR = "/cmlscratch/jkirchen/watermarking-root/eval_groups/baselines/core/detectgpt/clean/core_simple_1_50_200_gen"
# INPUT_DIR = OUTPUT_DIR = "/cmlscratch/jkirchen/watermarking-root/eval_groups/baselines/core/detectgpt/clean/core_simple_1_200_1000_gen"

# # note might have to skip full on this one idk
# INPUT_DIR = OUTPUT_DIR = "/cmlscratch/jkirchen/watermarking-root/eval_groups/baselines/human/detectgpt/clean/vicuna_lfqa_600_0-25_4-0_0-7_1-0_gen_filtered"

if "50_200" in INPUT_DIR:
    max_prefix_length = 200
elif "200_1000" in INPUT_DIR:
    max_prefix_length = 1000
else:
    print(f"Not sure if this is right, but setting max_prefix_length to 600")
    max_prefix_length = 600

# enumerate the contenst of the dir
dir_contents = os.listdir(INPUT_DIR)
# check whether the names contain "attacked" or not
if all(["attacked" in fname for fname in dir_contents]):
    out_jsonl_filename = "gen_table_attacked.jsonl"
    in_meta_filename = out_meta_filename = "gen_table_attacked_meta.json"
else:
    out_jsonl_filename = "gen_table.jsonl"
    in_meta_filename = out_meta_filename = "gen_table_meta.json"

# pop the meta file first
in_meta_file = f'{INPUT_DIR}/{dir_contents.pop(dir_contents.index(in_meta_filename))}'
# sort the rest of the files by the trailing integer after last '_' with "full" as the max value
dir_contents = sorted(dir_contents, key=lambda x: int(x.split("_")[-1].split(".")[0]) if x.split("_")[-1].split(".")[0] != "full" else 1000000)

# get the list of prefix lengths
prefix_lengths = [int(fname.split("_")[-1].split(".")[0]) if fname.split("_")[-1].split(".")[0] != "full" else max_prefix_length for fname in dir_contents]
                      
out_data_path = f"{INPUT_DIR}/{out_jsonl_filename}"
out_meta_path = f"{INPUT_DIR}/{out_meta_filename}"

print(dir_contents)
print(out_data_path)
print(out_meta_path)
print(prefix_lengths)
print(max_prefix_length)

['gen_table_attacked_50.jsonl', 'gen_table_attacked_100.jsonl', 'gen_table_attacked_200.jsonl', 'gen_table_attacked_full.jsonl']
/cmlscratch/jkirchen/watermarking-root/eval_groups/baselines/core/detectgpt/attacked/core_simple_1_200_1000_gen_cp_attack_3-25%/gen_table_attacked.jsonl
/cmlscratch/jkirchen/watermarking-root/eval_groups/baselines/core/detectgpt/attacked/core_simple_1_200_1000_gen_cp_attack_3-25%/gen_table_attacked_meta.json
[50, 100, 200, 1000]
1000


In [6]:
# concatenate all the jsonl files in order, and then write out to a single jsonl file with no integer or "full" suffix
# no need to load the meta file, it has same name and is already in place

all_data = []
# load the the files += each list of dicts
for prefix_len, fname in tqdm(zip(prefix_lengths,dir_contents)):
    fname_path = f'{INPUT_DIR}/{fname}'
    fname_data = [ex for ex in read_jsonlines(fname_path)]

    # if the "prefix_length" col is not in the data, add it using the max_prefix_length value set above
    if "prefix_length" not in fname_data[0]:
        for ex in fname_data:
            ex["prefix_length"] = max_prefix_length
        print(f"Added prefix_length col to data, eg. last rows prefix_length now: {fname_data[-1]['prefix_length']}")

    # check that the prefix_length col is consistent with the filename
    if not all([ex["prefix_length"] == prefix_len for ex in fname_data]):
        # update the prefix_length col to match the filename
        for ex in fname_data:
            ex["prefix_length"] = prefix_len
    assert all([ex["prefix_length"] == prefix_len for ex in fname_data]), f"prefix_length col is not consistent with filename: {fname_path}"

    all_data += fname_data

# show the unique prefix lengths now in the data
# using dict keys to kep order
found_lens = {}
for ex in all_data:
    found_lens[ex["prefix_length"]] = True
print(f"Found prefix lengths: {list(found_lens.keys())}")

# convert to a hf dataset
all_data = Dataset.from_list(all_data)

# print the length of the resulting list of dicts
all_data

Reading JSON lines from /cmlscratch/jkirchen/watermarking-root/eval_groups/baselines/core/detectgpt/attacked/core_simple_1_200_1000_gen_cp_attack_3-25%/gen_table_attacked_50.jsonl: 100%|██████████| 500/500 [00:00<00:00, 2133.34lines/s]
Reading JSON lines from /cmlscratch/jkirchen/watermarking-root/eval_groups/baselines/core/detectgpt/attacked/core_simple_1_200_1000_gen_cp_attack_3-25%/gen_table_attacked_100.jsonl: 100%|██████████| 500/500 [00:00<00:00, 2447.65lines/s]
Reading JSON lines from /cmlscratch/jkirchen/watermarking-root/eval_groups/baselines/core/detectgpt/attacked/core_simple_1_200_1000_gen_cp_attack_3-25%/gen_table_attacked_200.jsonl: 100%|██████████| 500/500 [00:00<00:00, 2360.46lines/s]
Reading JSON lines from /cmlscratch/jkirchen/watermarking-root/eval_groups/baselines/core/detectgpt/attacked/core_simple_1_200_1000_gen_cp_attack_3-25%/gen_table_attacked_full.jsonl: 100%|██████████| 500/500 [00:00<00:00, 1887.88lines/s]
4it [00:00,  4.01it/s]


Added prefix_length col to data, eg. last rows prefix_length now: 1000
Found prefix lengths: [50, 100, 200, 1000]


Dataset({
    features: ['idx', 'truncated_input', 'baseline_completion', 'orig_sample_length', 'prompt_length', 'baseline_completion_length', 'no_wm_output', 'w_wm_output', 'no_wm_output_length', 'w_wm_output_length', 'spike_entropies', 'baseline_completion_tokd', 'no_wm_output_tokd', 'w_wm_output_tokd', 'w_wm_output_attacked', 'w_wm_output_attacked_length', '__index_level_0__', 'prefix_length', 'baseline_completion_detectgpt_score_100_d', 'no_wm_output_detectgpt_score_100_d', 'baseline_completion_detectgpt_score_100_z', 'no_wm_output_detectgpt_score_100_z'],
    num_rows: 2000
})

In [7]:
# # write the file
# write_jsonlines(all_data, out_data_path)

Writing JSON lines at /cmlscratch/jkirchen/watermarking-root/eval_groups/baselines/core/detectgpt/attacked/core_simple_1_200_1000_gen_cp_attack_3-25%/gen_table_attacked.jsonl: 100%|██████████| 2000/2000 [00:04<00:00, 429.00it/s]


In [ ]:
# args = Namespace()
# in_metadata = out_metadata = read_json(in_meta_path)
# args.__dict__.update(in_metadata)

# raw_rows = [ex for ex in read_jsonlines(in_data_path)]
# in_ds = Dataset.from_list(raw_rows)
# in_ds

In [ ]:
# # load the idx filter data

# idx_jsonl_filename = "filtered_for_paraphrase_annotation_full.jsonl"

# idx_filter_data_dir = f"/cmlscratch/jkirchen/watermarking-root/eval_groups/ours/human/clean/evald_and_filtered_for_annotation"

# idx_filter_data_path = f"{idx_filter_data_dir}/{idx_jsonl_filename}"


In [ ]:
# raw_rows = [ex for ex in read_jsonlines(idx_filter_data_path)]
# idx_ds = Dataset.from_list(raw_rows)
# idx_ds

In [ ]:
# # make the out_run_dir if it doesn't exist
# os.makedirs(out_run_dir, exist_ok=True)

# # then write the filtered dataset out to disk as jsonl
# # and the metadata as json

# filtered_ds_lst = [ex for ex in filtered_ds]
# write_jsonlines(filtered_ds_lst, f"{out_run_dir}/{out_jsonl_filename}")

# write_json(out_metadata, f"{out_run_dir}/{out_meta_filename}",indent=4)